## Импорт и определение библиотек

In [1]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# ставим разные пакеты оттуда
!pip install langchain langchain-openai langchain-community openai tiktoken langchain-huggingface -q
!pip install beautifulsoup4 pypdf sentence-transformers faiss-cpu unstructured pypandoc -q
#huggingface_hub  langchain_experimental langchainhub
#!pip install unstructured "unstructured[pdf]" -q
!pip install sentence-transformers numpy transformers -q

In [ ]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
#from langchain.tools.retriever import create_retriever_tool
#from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool, initialize_agent
from langchain.prompts import PromptTemplate
import tqdm as notebook_tqdm
from huggingface_hub import HfApi

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# для использования ключа из LLM_bot
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py -q

# Готовим коктейль с помощью RAG
## Немного `Document Loader`

In [5]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub))
print('max chars on page:', max([len(page.page_content) for page in doc_epub]))

pages count: 1
max chars on page: 107261


In [43]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
"""loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))"""

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

 50%|█████     | 1/2 [02:05<02:05, 125.31s/it]


pages count: 481
max chars on page: 1854


'\n#!pip install unstructured "unstructured[pdf]" -q\nfrom langchain_community.document_loaders import DirectoryLoader\nloader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)\ndata = loader.load()'

## Исполним `Text splitters` и разобьём на чанки

In [20]:
# для gpt-3.5-turbo
splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],  # Разделители для сохранения структуры
    chunk_size=1000,  # 1000 символов (~200-250 токенов)
    chunk_overlap=200  # 200 символов перекрытия
)

# для gpt-4o-mini
"""splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],
    chunk_size=2000,  # 2000 символов (~400-500 токенов)
    chunk_overlap=300  # 300 символов перекрытия
)"""

# Разделяем книгу на части
#documents = splitter.split_documents(doc_epub)

split_documents = []
for page in doc_epub:
    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

len(split_documents)

'split_documents = []\nfor page in data:\n    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])\n\nlen(split_documents)'

In [21]:
chunks = splitter.split_documents(doc_epub)
print('Количество чанков:', len(chunks))
print('Максимальное количество символов в чанке:', max([len(chunk.page_content) for chunk in chunks]))

Количество чанков: 167
Максимальное количество символов в чанке: 998


## Чутка `embeddings с HuggingFace`

/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Чтение токена из файла
def read_token(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
token = read_token('../utils.key')

In [ ]:
# Формирование эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  
    #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
    #"cointegrated/LaBSE-en-ru",
    model_kwargs={
        "device": "cpu",
        "token": token
                  } 
)

## Наполним чашу `Vector Store`

In [12]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            split_documents, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_epub")

# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_epub", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [119]:
"""query = "грейпфрутовый сок, ржаной виски"
results = db_embed.similarity_search(query, k=5)
for result in results:
    print(result.page_content)"""

'query = "грейпфрутовый сок, ржаной виски"\nresults = db_embed.similarity_search(query, k=5)\nfor result in results:\n    print(result.page_content)'

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [93]:
# Если используете ключ из курса по LLM, запустите эту ячейку
from utils import ChatOpenAI

# Считываем API ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            course_api_key = line.split('=')[1].strip()
            break

# Инициализация языковой модели
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",         #"gpt-4o-mini", 
    temperature=0.2, 
    course_api_key=course_api_key
)

# объявление retriever 
retriever = db_embed.as_retriever(
        search_type="mmr",              # тип поиска похожих документов
        search_kwargs={
                "k": 5,                 # количество возвращаемых документов (Default: 4)
                "fetch_k": 20
            },     
    )


retrieval_qa = RetrievalQA.from_chain_type(
        llm=llm, 
        retriever=retriever,
        return_source_documents=True    # Возвращает исходные документы
    )

# Месье системный промт

In [95]:
# Системный промт для агента

system_prompt = """
        Ты — помощник для поиска рецептов коктейлей. У тебя есть доступ к базе данных с рецептами, вдохновлёнными литературными произведениями.
        Когда пользователь вводит ингредиенты, используй инструмент "Cocktail Recipe Finder", чтобы найти подходящие рецепты.
        Если рецепт не найден, предложи альтернативные варианты или уточни запрос.

        Пример:
        Пользователь: Какие коктейли можно сделать из водки и апельсинового сока?
        Агент: Использую инструмент "Cocktail Recipe Finder" для поиска рецептов. Вот что найдено: [рецепт].
    """

# Миксуем рецепт коктейля `agent` 00х

In [ ]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Используй этот инструмент, чтобы найти рецепты коктейлей по заданным ингредиентам. Вводи список ингредиентов, и инструмент вернёт подходящие рецепты. Пример: ['сок', 'виски']."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="conversational-react-description",       #"zero-shot-react-description", "plan-and-execute"
    verbose=True,
    system_prompt=system_prompt                     # не забыть указать системный промт для вкуса
)

# Задаём общие параметры, после чего коктейль готов

In [120]:
# Объявление функции на ввод данных от пользователя

def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients

# Обработка ответа
def format_response(response):
    if "не знаю" in response.lower() or "не найдено" in response.lower():
        return "К сожалению, я не нашёл коктейлей с этими ингредиентами. Попробуйте другие ингредиенты."
    else:
        return response


In [141]:
# Функция выбора стиля ответа

def choose_style():
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "по-умолчанию"  # Стиль по умолчанию
    }
    #print("Доступные стили ответа:")
    for key, value in styles.items():
        print(f"{key}: {value}")
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "style4")

"""style = choose_style()
prompt = PromptTemplate(input_variables=['output_text', 'style'],
                        template='''Перепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:''')
style_changer_chain = prompt | llm

styled_response = style_changer_chain.invoke({'output_text': response, 'style': style}).content
print("\nОтвет в выбранном стиле:")
print(styled_response)"""

'style = choose_style()\nprompt = PromptTemplate(input_variables=[\'output_text\', \'style\'],\n                        template=\'\'\'Перепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:\'\'\')\nstyle_changer_chain = prompt | llm\n\nstyled_response = style_changer_chain.invoke({\'output_text\': response, \'style\': style}).content\nprint("\nОтвет в выбранном стиле:")\nprint(styled_response)'

In [ ]:
"""# Формирование запроса и вывод результата

def main():
    print("Добро пожаловать в поиск коктейлей!")
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Преобразуем массив ингредиентов в строку
        ingredients_str = ", ".join(ingredients)
        
        # Формируем текстовый запрос
        user_prompt = (
            """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
            Для каждого коктейля опиши подробный способ приготовления и укажи пропорции. 
            Отвечай строго на русском языке, используя чёткий и лаконичный формат. 
            Не добавляй лишнюю информацию, которая не относится к рецепту. 
            Ингредиенты: {ingredients}""".format(ingredients=ingredients_str)
        )

        print("Запрашиваю информацию...")
        response = retrieval_qa.invoke({"query": user_prompt})  
        
        # Отладочный вывод
        print("\nРезультат поиска:")
        print(response["result"])

        print("Спасибо за использование поиска коктейлей! До свидания!")
        break

if __name__ == "__main__":
    main()
"""

Добро пожаловать в поиск коктейлей!
Запрашиваю информацию...

Результат поиска:
Из представленных ингредиентов можно приготовить коктейль "Грейпфрутовый виски".

**Коктейль "Грейпфрутовый виски"**

**Ингредиенты:**
- 60 мл ржаного виски
- 90 мл грейпфрутового сока

**Приготовление:**
1. В шейкер добавьте ржаной виски и грейпфрутовый сок.
2. Добавьте лед в шейкер.
3. Закройте шейкер и тщательно взболтайте в течение 15-20 секунд.
4. Процедите коктейль в охлажденный стакан.

Подавайте сразу.
Спасибо за использование поиска коктейлей! До свидания!


In [143]:
def main():
    #print("Добро пожаловать в поиск коктейлей!")
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Преобразуем массив ингредиентов в строку
        ingredients_str = ", ".join(ingredients)
        
        # Формируем текстовый запрос
        user_prompt = (
            """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
            Для каждого коктейля опиши подробный способ приготовления и укажи пропорции. 
            Отвечай строго на русском языке, используя чёткий и лаконичный формат. 
            Не добавляй лишнюю информацию, которая не относится к рецепту. 
            Ингредиенты: {ingredients}""".format(ingredients=ingredients_str)
        )

        response = retrieval_qa.invoke({"query": user_prompt})  
        
        # Получаем ответ без вывода его на экран
        original_response = response["result"]

        # Выбор стиля
        style = choose_style()
        prompt = PromptTemplate(input_variables=['output_text', 'style'],
                                template='''Добро пожаловать в поиск коктейлей!\nПерепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:\n''')
        
        # Генерация текста с использованием оригинального ответа и выбранного стиля
        styled_prompt = prompt.format(output_text=original_response, style=style)
        
        # Применение выбранного стиля к оригинальному ответу
        styled_response = llm.invoke(styled_prompt).content
        
        # Переформулировка заголовков
        styled_response = styled_response.replace("Коктейль:", "Название коктейля:") 
        styled_response = styled_response.replace("Ингредиенты:", "Состав:") 
        styled_response = styled_response.replace("Приготовление:", "Способ приготовления:")
        
        # Добавление финальной фразы
        #final_response = styled_response + "\nСпасибо за использование поиска коктейлей! До свидания!"

        print(styled_response)
        break

if __name__ == "__main__":
    main()

1: стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта
2: гопническо-быдляцкий жаргон
3: экспериментальный стиль нарезок Уильяма Берроуза
4: по-умолчанию
Коктейль 1: Виски-кола

Состав:
- 50 мл виски, темного и загадочного, как бездны космоса
- 150 мл колы, шипучей и зловещей, как шепоты древних существ
- Лед по вкусу, холодный, как дыхание мертвецов

Способ приготовления:
1. Наполните стакан льдом, словно заполняя его осколками ледяных звезд.
2. Влейте виски, погружая в него мрак, который пробуждает забытые страхи.
3. Добавьте колу, наблюдая, как она бурлит, как безумные мысли в умах безумцев.
4. Аккуратно перемешайте, позволяя темным силам соединиться, и подавайте, как жертву древним богам.

Коктейль 2: Виски с соком

Состав:
- 50 мл виски, пронизанного тайнами и ужасами
- 100 мл апельсинового сока, яркого, как вспышка безумия в ночи
- Лед по вкусу, как холодные объятия забвения

Способ приготовления:
1. Наполните стакан льдом, словно заполняя его холодом, который прон

In [32]:
"""## Если не хотите платить денежков, то запустите эту ячейку
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceEmbeddings

# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('HUGGINGFACEHUB_API_TOKEN'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# Создаем объект языкового модели
llm = HuggingFaceEndpoint(
      #repo_id="IlyaGusev/saiga_llama3_8b",
      repo_id="mistralai/Mistral-7B-Instruct-v0.3",
      task="text-generation",  # Вид задачи, в нашем случае - генерация текста
      huggingfacehub_api_token = course_api_key
    )"""

# Разные способы поиска цен на ингредиенты

## Поиск с помощью агента
1. Инструменты LangChain:
    - Мы можем использовать инструменты, такие как `DuckDuckGoSearchRun` (для поиска через DuckDuckGo), или создать собственные инструменты для взаимодействия с API магазинов (например, Winelab, SimpleWine и других).
2. Агент LangChain:
    - Агент будет обрабатывать запросы, отправлять их в поисковые инструменты и агрегировать ответы.
3. Поиск стоимости товара:
    - Агент использует инструменты, чтобы найти релевантную информацию о стоимости алкогольных ингредиентов.

In [ ]:
pip install -U duckduckgo-search -q

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI

# Подключение LLM
#llm = OpenAI(temperature=0.0, openai_api_key=course_api_key)

# Создаем поисковый инструмент DuckDuckGo
search = DuckDuckGoSearchRun()

# Добавляем инструмент в список
tools = [
    Tool(
        name="Search ingredient",
        func=search.run,
        description="Используйте этот инструмент для поиска информации об ингредиентах и ценах на самые популярные из них."
    )
]

# Инициализация агента
price_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",  # Используем Zero-shot агент
    verbose=True
)

# Формируем промт для поиска цен на товар
ingredient = "бурбон"
city = "Санкт-Петербург"
prompt = (
    f"Найдите цену на продукт '{ingredient}' в интернет-магазинах {city}. "
    """Если вариантов несколько, укажите 3 лучших результата. 
    Если в ответе название пишется на языке производителя, то оставь как есть. Название валюты и объём должны быть указаны на русском языке. 
    Укажи ответ в формате: название экземпляра, стоимость в среднем ценовом диапазоне, объём, ссылка на магазин, если имеется; если нет, то ничего не писать. 
    Если вариантов ответа несколько, то отобрази их в виде вертикального списка, где каждый новый вариант отображается с новой строки. """
)

# Выполнение агента
result = price_agent.run(prompt)

print(result)


## Поиск с помощью `RAG`

Общая стратегия:

1. Использование `WebLoader` для сбора данных:
   - скачиваем страницы для каждой категории алкоголя с сайтов, таких как `Winelab`.
   - сохраняем их текстовое содержимое (например, названия и цены продуктов) в векторное хранилище.
1. Создание векторного хранилища:
    - разбиваем загруженные данные на чанки.
    - индексируем их с помощью векторизатора (например, FAISS или Chroma).
1. Поиск ингредиентов:
    - при запросе пользователя (например, "водка"), извлекаем соответствующую информацию из хранилища.
1. Использование `LangChain`:
    - для загрузки данных используем WebBaseLoader из LangChain.
    - для индексации данных — инструменты LangChain, такие как Chroma или FAISS.
    - для обработки запросов пользователя — цепочки, основанные на LLM (например, `OpenAI API`).

In [ ]:
pip install bs4 langchain-openai faiss-cpu -q

In [ ]:
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Установите ваш ключ API
# Считываем ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            # Извлекаем ключ, убирая лишние пробелы и символы
            course_api_key = line.split('=')[1].strip()
            break

# 1. Сбор данных с категорий сайта
def fetch_and_index_alcohol_data():
    categories = {
        "absinthe": "https://www.winelab.ru/catalog/krepkiy-alkogol-absent",
        "vodka": "https://www.winelab.ru/catalog/krepkiy-alkogol-vodka",
        "whiskey": "https://www.winelab.ru/catalog/krepkiy-alkogol-viski",
    }

    documents = []
    for category, url in categories.items():
        try:
            loader = WebBaseLoader(url)
            category_docs = loader.load()
            documents.extend(category_docs)
            print(f"Загружено {len(category_docs)} документов из {url}.")
        except Exception as e:
            print(f"Ошибка при загрузке данных из {url}: {e}")

    print(f"Всего загружено документов: {len(documents)}")
    return documents

# 2. Разбивка текста на чанки
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    print(f"Всего создано чанков: {len(chunks)}")
    return chunks


# 3. Индексация данных с использованием FAISS
def create_faiss_index(chunks):
    if not chunks:
        raise ValueError("Нет данных для индексации. Проверьте загрузку и разбивку документов.")
    
    embeddings = OpenAIEmbeddings(openai_api_key=course_api_key)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore


# 4. Создание цепочки RAG для поиска
def create_retrieval_chain(vectorstore):
    retriever = vectorstore.as_retriever()
    llm = OpenAI(temperature=0.0, openai_api_key=course_api_key)
    chain = RetrievalQA(llm=llm, retriever=retriever)
    return chain

# Основная функция
def main():
    print("Сбор данных...")
    documents = fetch_and_index_alcohol_data()
    print("Разделение на чанки...")
    chunks = split_documents(documents)
    print("Создание векторного индекса...")
    vectorstore = create_faiss_index(chunks)
    print("Создание цепочки RAG...")
    rag_chain = create_retrieval_chain(vectorstore)

    # Поиск цены ингредиента
    ingredient_query = "водка"
    print(f"Поиск для: {ingredient_query}")
    result = rag_chain.run(ingredient_query)
    print(f"Результат: {result}")

if __name__ == "__main__":
    main()

## Поиск `в лоб`

In [ ]:
import requests
import re

def search_price_with_specific_cookies(ingredient):
    """
    Функция для поиска цен на алкогольные ингредиенты с индивидуальными куки для каждого магазина.
    """
    stores = [
        {
            "name": "Winelab",
            "url": lambda ing: f"https://www.winelab.ru/search/facets?text={ing}",
            "cookies": {"age-confirmed": "1"},
            "title_pattern": r'<a class="product-card__link" .*? title="(.*?)"',
            "price_pattern": r'<div class="price__main-value">(.*?)</div>',
            "link_pattern": r'<a class="product-card__link" href="(.*?)"',
            "link_prefix": "https://www.winelab.ru"
        },
        {
            "name": "SimpleWine",
            "url": lambda ing: f"https://simplewine.ru/catalog/?q={ing}",
            "cookies": {"age-confirm": "1"},
            "title_pattern": r'<a class="product-card__title-link" .*?>(.*?)</a>',
            "price_pattern": r'<div class="product-card__price-current">(.*?)</div>',
            "link_pattern": r'<a class="product-card__title-link" href="(.*?)"',
            "link_prefix": "https://simplewine.ru"
        }
    ]

    results = []

    for store in stores:
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"
            }
            response = requests.get(store["url"](ingredient), headers=headers, cookies=store["cookies"], timeout=5)
            if response.status_code == 200:
                # Извлечение данных о товарах
                titles = re.findall(store["title_pattern"], response.text)
                prices = re.findall(store["price_pattern"], response.text)
                links = re.findall(store["link_pattern"], response.text)

                for title, price, link in zip(titles[:3], prices[:3], links[:3]):
                    full_link = f"{store['link_prefix']}{link}"
                    results.append(f"- {title.strip()} {price.strip()} рублей {full_link} ({store['name']})")

                if results:
                    break  # Если найдены результаты, завершаем поиск
            else:
                results.append(f"Ошибка доступа к {store['name']} для {ingredient}: статус {response.status_code}")
        except Exception as e:
            results.append(f"Ошибка при выполнении поиска в {store['name']} для {ingredient}: {str(e)}")

    if not results:
        results.append(f"Данные о наличии в продаже {ingredient} не найдены.")

    return "\n".join(results)

# Пример использования
ingredient_name = "джин"
print(search_price_with_specific_cookies(ingredient_name))

In [ ]:
# Разбор коктейля и ингредиентов для поиска цен
pattern = r"\*\*(.*?)\*\*:.*?((?:\n\s*-\s*.*?\n)+)"
match = re.search(pattern, response, re.DOTALL)
if match:
    cocktail_name = match.group(1)
    ingredients_section = match.group(2)
    print(f"\nКоктейль: {cocktail_name}")
    print(f"Ингредиенты:")
    print(ingredients_section)
    
    # Поиск цен для каждого ингредиента
    ingredient_lines = ingredients_section.strip().split("\n")
    for line in ingredient_lines:
        ingredient_match = re.search(r"-\s*\d+\s*мл\s*([^\n]+)", line)
        if ingredient_match:
            ingredient_name = ingredient_match.group(1)
            print(f"\nИщем цену для: {ingredient_name}")
            prices = search_price(ingredient_name)
            print(prices)
        else:
            print(f"\nНе удалось извлечь название ингредиента из строки: {line}")
else:
    print("Не удалось найти информацию о коктейле и ингредиентах в ответе модели.")

## Поиск с помощью Selenium

In [ ]:
pip install selenium -q

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def search_price_with_selenium(ingredient):
    """
    Поиск цен на ингредиенты с использованием Selenium.
    """
    # Укажите путь к вашему WebDriver
    service = Service('/home/elijah/Documents/chromedriver-linux64/chromedriver')  # Укажите корректный путь к ChromeDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Запуск в безголовом режиме
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(service=service, options=chrome_options)

    results = []
    try:
        # Магазин Winelаб
        driver.get("https://www.winelab.ru")

        # Подтверждение возраста
        age_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "age-confirm"))
        )
        age_button.click()

        # Поиск ингредиента
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "text"))  # Обратите внимание на имя поля
        )

        # Добавление небольшой задержки для обеспечения доступности элемента
        time.sleep(1)

        # Используем JavaScript для установки значения в поле ввода
        driver.execute_script("arguments[0].value = arguments[1];", search_box, ingredient)
        
        # Теперь вызываем событие input для имитации ввода текста
        search_box.send_keys(Keys.RETURN)

        # Найдите кнопку отправки и нажмите ее (если это необходимо)
        # search_button = driver.find_element(By.CLASS_NAME, "js_search_button")
        # search_button.click()

        # Сбор результатов
        products = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "product-card"))
        )
        
        for product in products[:3]:  # Берём максимум 3 результата
            title = product.find_element(By.CLASS_NAME, "product-card__title").text
            price = product.find_element(By.CLASS_NAME, "price__main-value").text
            
            results.append(f"- {title} {price.strip()} рублей")

    except Exception as e:
        results.append(f"Ошибка при обработке ингредиента {ingredient}: {str(e)}")
    finally:
        driver.quit()

    return results if results else f"Данные о наличии в продаже {ingredient} не найдены."

# Пример использования
ingredient_name = "водка"
print(search_price_with_selenium(ingredient_name))